In [111]:
import torch
from torch.nn import GRU
from molNet.dataloader.base_loader import df_to_generator, GeneratorDataLoader, GeneratorDataset
from molNet.featurizer.featurizer import OneHotFeaturizer
from molNet.mol.molecules import Molecule
%load_ext autoreload
import pandas as pd
from molNet.dataloader.molecule_loader import MoleculeGraphFromGeneratorLoader, SmilesFromGeneratorLoader, \
    MoleculeFromGeneratorLoader, MoleculeLoaderMixin
%autoreload 2

[autoreload of molNet.dataloader.base_loader failed: Traceback (most recent call last):
  File "/home/julian/anaconda3/envs/molNet/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/julian/anaconda3/envs/molNet/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/home/julian/anaconda3/envs/molNet/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/home/julian/anaconda3/envs/molNet/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 302, in update_class
    if update_generic(old_obj, new_obj): continue
  File "/home/julian/anaconda3/envs/molNet/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/home/julian/anaconda3/envs/molNet/lib/python3.7/site-packages/IPython/extensions/

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [101]:
df = pd.read_csv("../../datasets/chembl_27_chemreps.txt.gz",sep="\t")

In [ ]:


sg = MoleculeFromGeneratorLoader(*df_to_generator(df,processing = lambda d:(d['canonical_smiles'],{})))

sg.prepare_data()
sg.setup()
chars=set()
b=0
m=0
tl=len(sg.train_dataloader())
for batch in sg.train_dataloader():
    mol: Molecule
    if b > 10:
        break
    print("b {}/{}".format(b,tl))
    b+=1
    for mol in batch:
        print("m",m,end="\r")
        m+=1
        chars.update({s for smiles in mol.get_random_smiles() for s in smiles})
chars

B 1358987
b 0/42469
b 1/42469
b 2/42469
b 3/42469
b 4/42469
b 5/42469


In [102]:
smiles_one_hot = OneHotFeaturizer(
    possible_values=["START",'#', '%', '(', ')', '+', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '=', '@', 'A', 'B', 'C', 'F', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'X', 'Z', '[', '\\', ']', 'a', 'b', 'c', 'e', 'g', 'i', 'l', 'n', 'o', 'p', 'r', 's', 't',"END"],
    name="smiles_one_hot"
)

In [103]:
chars

['START',
 'C',
 'c',
 '1',
 'c',
 'c',
 '(',
 '-',
 'c',
 '2',
 'c',
 's',
 'c',
 '(',
 'N',
 '=',
 'C',
 '(',
 'N',
 ')',
 'N',
 ')',
 'n',
 '2',
 ')',
 'c',
 'n',
 '1',
 'C',
 'END']

In [14]:
smile_chars=set()
for r,d in df.iterrows():
    print(r,end="\r")
    try:
        smile_chars.update(set(d['canonical_smiles']))
    except:
        pass
smile_chars

{'#',
 '%',
 '(',
 ')',
 '+',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '=',
 '@',
 'A',
 'B',
 'C',
 'F',
 'H',
 'I',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'R',
 'S',
 'T',
 'X',
 'Z',
 '[',
 '\\',
 ']',
 'a',
 'b',
 'c',
 'e',
 'g',
 'i',
 'l',
 'n',
 'o',
 'p',
 'r',
 's',
 't'}

In [15]:
smile_chars.update(chars)

In [7]:
print(sorted(smile_chars))

NameError: name 'smile_chars' is not defined

In [12]:
import numpy as np
smiles=df['canonical_smiles'][0]
chars=["START"]+list(smiles)+["END"]
np.array([smiles_one_hot(s) for s in chars])

array([[ True, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False,  True]])

RuntimeError: each element in list of batch should be of equal size

In [238]:
all_smiles_string=[]

class SmilesOneHotgenerator():
    def __init__(self, smiles_list,max_length=None):
        self.smiles_list = smiles_list
        self._length = sum(len(s) for s in self.smiles_list)*10
        if max_length is None:
            max_length = max(len(s) for s in self.smiles_list)*3
        self.max_length=max_length
        def d():
            shape=(self.max_length,len(smiles_one_hot))
            ots={"START":(smiles_one_hot("START"),np.argmax(smiles_one_hot("START"))),
                 "END":(smiles_one_hot("END"),np.argmax(smiles_one_hot("END")))
                 }
            seq_length=100
            sequence=np.zeros((seq_length,len(smiles_one_hot)))
            target=np.zeros(seq_length)

            i=0
            pre_char="END"
            #target[i]=ots[pre_char][1]
            #i+=1
            for mol_smile in self.smiles_list:
                for smiles in Molecule.from_smiles(mol_smile).get_random_smiles(1):
                    for c in ["START"]+list(smiles)+["END"]:
                        if c not in ots:
                            ots[c] = (smiles_one_hot(c),np.argmax(smiles_one_hot(c)))

                        sequence[i]=ots[pre_char][0]
                        target[i]=ots[c][1]
                        pre_char=c
                        i+=1
                        if i == seq_length:
                            yield torch.from_numpy(sequence).float(), torch.from_numpy(target).long()
                            i=0

        #    for i,c in enumerate(list(smiles)):
        #        if c not in ots:
        #            ots[c] = torch.Tensor(smiles_one_hot(c)).float()
        #        yield ots[pre_char],torch.argmax(ots[c]).long()
        #        pre_char = c
        #    yield ots[pre_char],torch.argmax(ots["END"]).long()
        #    yield ots["END"],torch.argmax(ots["START"]).long()

        self.iter = d()

    def __len__(self):
        return self._length

    def __next__(self):
        return next(self.iter)

len(SmilesOneHotgenerator(df['canonical_smiles'].dropna()))

1110152900

In [246]:
loader = GeneratorDataLoader(generator=SmilesOneHotgenerator(df['canonical_smiles'].dropna()),batch_size=3)

In [258]:
import pytorch_lightning as pl
class SmilesEncoderDecoder(pl.LightningModule):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.gru = GRU(len(smiles_one_hot),len(smiles_one_hot),5,batch_first=True)

        self.loss = torch.nn.CrossEntropyLoss()

    def forward(self,x):
        x,h=self.gru(x)
        return x

    def training_step(self,batch, *args, **kwargs):
        x,y=batch
        #print(x.shape)
        y_hat=self(x)
        #rint(y_hat.shape,y.shape)
        loss = self.loss(y_hat.reshape(y_hat.size(0)*y_hat.size(1),-1),y.reshape(y_hat.size(0)*y_hat.size(1)).long())
        result = pl.TrainResult(minimize=loss)#
        result.log('train_loss', loss, on_epoch=True)
        return result

    def configure_optimizers(self):
            optimzer = torch.optim.AdamW(self.parameters(), lr=0.001,amsgrad=True)
            return optimzer

model = SmilesEncoderDecoder()

trainer = pl.Trainer(max_epochs=100000,gpus=0)

trainer.fit(model,loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name | Type             | Params
------------------------------------------
0 | gru  | GRU              | 95 K  
1 | loss | CrossEntropyLoss | 0     
Saving latest checkpoint..


1

In [ ]:

ize (3, 56), got torch.Size([3, 100])